In [2]:
import numpy as np

### In this file we will go through the Devitt el al. paper (arXiv: 0905.2794v4) and derive and explain all of the math and theory to fully understand basic QEC

## ll. Preliminaries

### This section describes:
    (a) The basic structure of a qubit
    (b) Some general requirements of QEC

#### Suggest reading this section of the paper to understand the fundamentals of qubits and why we need QEC

In [3]:
# Pauli Operators
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,1j],[-1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
sigma_I = np.identity(2)
print('Below we show the Pauli Operators which can be used to describe any quantum gate on an individual qubit:')
print('Pauli-X: \n', sigma_x) 
print('Pauli-Y: \n', sigma_y)
print('Pauli-Z: \n', sigma_z)
print('Pauli-I: \n', sigma_I)

Below we show the Pauli Operators which can be used to describe any quantum gate on an individual qubit:
Pauli-X: 
 [[0 1]
 [1 0]]
Pauli-Y: 
 [[ 0.+0.j  0.+1.j]
 [-0.-1.j  0.+0.j]]
Pauli-Z: 
 [[ 1  0]
 [ 0 -1]]
Pauli-I: 
 [[1. 0.]
 [0. 1.]]


## lll. Quantum Errors: Cause and Effect

### In this section we consider different sources of error 
#### (coherent, environmental decoherence, and loss, leakage, measuremnt and initialization)
### We will apply these errors to two different algorithms.

### 1. A single qubit undergoing N identity opertations
$$\vert\psi_{final}\rangle = \prod_{i}^{N} I_{i} \vert0\rangle = \vert0\rangle $$ Where $I \equiv \sigma_{I}$

#### If a qubit is initially in the $\vert0\rangle $state then this operation in the basis $\vert0\rangle$, $\vert1\rangle$ would be $\vert0\rangle$
#### This can be shown with the code below:
#### Feel free to play with the initial state values and see what results :)

In [4]:
initial_state = np.array([1, 0]) # initalize initial state to |0>
n = 10 # number of times to apply the I operator (can be anything)
I_i = np.identity(2) # initialize I_i for i = 1 first

# Since we are only indexing over i we can multiply I_i with sigma_I n times to arrive at I_n final
for i in range(0,n):
    I_n = np.dot(I_i, sigma_I)

final_state = np.dot(I_n, initial_state)
print('Here we can see that the final state is |0> ')
print('Final State: ', final_state)

Here we can see that the final state is |0> 
Final State:  [1. 0.]


### 2. An algorithm of 3 gates acting on a qubit
$$\vert\psi\rangle = HIH\vert0\rangle = HI\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle) = H\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle) = \vert0\rangle$$
#### We can show the above case is correct by applying this algorithm to a single qubit in the $\vert0\rangle$ state:
###### Feel free to play with the initial state values and see what results :)

In [6]:
initial_state = np.array([1,0]) # initalize initial state to |0>
h = 1/np.sqrt(2)*np.array([[1,1],[1,-1]]) # set the hadamard operator 

# Below we perform the 3 gate operation and psi just represents the state of the qubit during the operations
psi = np.dot(h, initial_state) # apply the first hadamard gate
psi = np.dot(sigma_I, psi) # apply a wait stage with the I gate
final_state = np.dot(h, psi) # apply the second hadamard gate

print('Here we can see that the final state is |0> ')
print('Final State: ', (np.rint(final_state)).astype(int))

Here we can see that the final state is |0> 
Final State:  [1 0]


### A. Coherent Quantum Errors: Gates which are incorrectly applied:
#### This error is usually related to incorrect knowledge of the system dynamics, but it is coherent

### First we will consider an example which applies an incorrect rotation about the X-axis of the Bloch Sphere
$$ \vert\psi_{final}\rangle = \prod_{k}^{N} e^{i\epsilon\sigma_{x}}\vert0\rangle = \cos(N\epsilon)\vert0\rangle + i\sin(N\epsilon)\vert1\rangle$$

#### Now we will measure the system in the $\vert0\rangle$, $\vert1\rangle$ basis
#### Due to the coherent quantum errors below is the probability of measuing the system in $\vert0\rangle$ or $\vert1\rangle$:
$$ P(\vert0\rangle) = \cos^2(N\epsilon) \approx 1 - (N\epsilon)^2 $$
$$ P(\vert1\rangle) = \sin^2(N\epsilon) \approx (N\epsilon)^2$$
#### Thus the probability of error is 
$$ p_{error} \approx (N\epsilon)^2$$
### Below we can see a coherent quantum error acting on the initial state with arbitrary $\epsilon$ and calculate the error due to the X-rotation error N times

In [7]:
initial_state = np.array([1,0]) # initalize initial state to |0>
epsilon = 10 # arbitrary epsilon is set
n = 10 # arbitrary N is set
# We can convert the exponential to its trigonometric form using euler's identity and some algebreic manipulation
final_state = np.dot(np.cos(n*epsilon), initial_state) + np.dot(1j*np.sin(n*epsilon), np.array([0,1]))
print('Final State: ', final_state)

Final State:  [0.86231887+0.j         0.        -0.50636564j]


#### We know that the probability of being in a certain state say $\vert0\rangle$ is $P(\vert0\rangle) = \alpha^2$ and for $\vert1\rangle$ is $P(\vert1\rangle) = \beta^2$ when $\vert\psi\rangle = \alpha\vert0\rangle + \beta\vert1\rangle$



In [8]:
prob_zero = (final_state[0].real + final_state[0].imag)**2 # Probability of being in the |0> state
prob_one = (final_state[1].real + final_state[1].imag)**2 # Probability of being in the |1> state

print('Probability of |0> State: ', prob_zero, '\nProbability of |1> State: ', prob_one)
print('We can see when adding the two probabilities we get one: ', (np.rint(prob_zero + prob_one).real).astype(int))

Probability of |0> State:  0.7435938375035028 
Probability of |1> State:  0.25640616249649706
We can see when adding the two probabilities we get one:  1


In [10]:
#### Calculating the probability for error: (This is the same as the P(|1>) or 1 - P(|0>))
prob_error = prob_one
print('Probability of error: prob_one = ', prob_error, ' = 1 - prob_zero = ', 1 - prob_zero)

Probability of error: prob_one =  0.25640616249649706  = 1 - prob_zero =  0.25640616249649717


### B. Environmental Decoherence:
#### This type of error usually stems from noise, heat, or other external factors

#### In this example we will focus on an environment which is a two level system and has two basis states: $\vert e_0\rangle$ and $ \vert e_1\rangle$
#### These satisfy the relations $ \langle e_i\vert e_j\rangle = \delta_{ij}$ and $ \vert e_0\rangle\langle e_0\vert + \vert e_1\rangle\langle e_1\vert = I$

#### We will assume 2 things:
##### 1. The environment coupling works as follows: when the qubit is in the $\vert1\rangle$ state the coupling flips the environment state, and when the qubit is in the $\vert0\rangle$ state nothing happens
##### 2. The system/environemnt only interact during the wait stage when the identity operator is acting

#### Let us assume the environment is initialized to the state $\vert E\rangle = \vert e_0\rangle$ and lets couple it to the 2nd system we considered in part lll which applies 3 gates on the initial state:
$$ HIH\vert0\rangle\vert E\rangle = \frac{1}{2}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle + \frac{1}{2}(\vert0\rangle - \vert1\rangle)\vert e_1\rangle$$

#### This can be simplified to the following since the error operator flips when acting on $\vert1\rangle$:
$$ HIH\vert0\rangle\vert E\rangle = HI\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle 
= H\frac{1}{\sqrt{2}}(\vert0\rangle\vert e_0\rangle + \vert1\rangle\vert e_1\rangle)
=\frac{1}{2}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle + \frac{1}{2}(\vert0\rangle - \vert1\rangle)\vert e_1\rangle$$

In [15]:
# We can show that this is the case below: 
# (note that any of the pauli operators would work in this case due to our assumptions)
initial_state = np.array([1,0]) # initalize initial state to |0>
h = 1/np.sqrt(2)*np.array([[1,1],[1,-1]]) # set the hadamard operator 
env_error_operator = sigma_y # set the environmental error operator

# Below we perform the 3 gate operation and psi just represents the state of the qubit during the operations
psi = np.dot(h, initial_state) # apply the first hadamard gate
psi = np.dot(np.dot(sigma_I, env_error_operator), psi) # apply a wait stage with the I gate
final_state = np.dot(h, psi) # apply the second hadamard gate

print('Here we can see the final state after the environmental decoherence: ')
print('Final State: ', final_state)

Here we can see the final state after the environmental decoherence: 
Final State:  [0.-2.23711432e-17j 0.+1.00000000e+00j]


#### However, now since we are considering decoherent operations, we are no longer in a pure state. Instead we have turned our quantum states into a classical mixed state, and we must move to a density matrix to be able to represent it. This matrix represents the lack of knowledge of the system, being a description of the system at all possible measurement results. 

#### We know that the density matix for a given state is $ p_f = \sum_{j}^{}{p_j\vert\psi_j\rangle\langle\psi_j\vert}$ where $p_j$ is the probability of being in the state $\vert\psi_j\rangle$. After plugging in and algebra we arrive at the density matrix:
$$ \rho_f = \frac{1}{4}(\vert0\rangle\langle0\vert + \vert0\rangle\langle1\vert + \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert)\vert e_0\rangle\langle e_0\vert
+ \frac{1}{4}(\vert0\rangle\langle0\vert - \vert0\rangle\langle1\vert - \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert)\vert e_1\rangle\langle e_1\vert$$
$$ + \frac{1}{4}(\vert0\rangle\langle0\vert - \vert0\rangle\langle1\vert + \vert1\rangle\langle0\vert - \vert1\rangle\langle1\vert)\vert e_0\rangle\langle e_1\vert
+ \frac{1}{4}(\vert0\rangle\langle0\vert + \vert0\rangle\langle1\vert - \vert1\rangle\langle0\vert - \vert1\rangle\langle1\vert)\vert e_1\rangle\langle e_0\vert $$

#### Since we dont know the environmental degrees of freedom we will trace over this part of the system (also known as a partial trace: https://en.wikipedia.org/wiki/Partial_trace#Partial_trace_for_operators_on_Hilbert_spaces)
$$ Tr_W\left(\sum_{k,l}^{}{T^{(kl)} \otimes \vert k\rangle\langle l\vert}\right) = \sum_{j}^{}{T^{(jj)}}$$ 
#### In our case, we have taken $\sum_{k,l}^{}{T^{(kl)}}$ as $\rho_f$ and W as E, which is our error
#### Therefore, if we take the trace over the error of the system, E, we will only be adding the parts of $p_f$ which contain $ \vert e_0\rangle\langle e_0\vert$ and $\vert e_1\rangle\langle e_1\vert$

### C. Simple models of loss, leakage, measurement and initialization

#### In this section, important & frequently used models will be discerned to identify Quantum Errors.

#### Measurement Errors - Describing in two ways

#### Description 1: Positive Operator Value Measures (POVM's)

$$F_0 = \left ( 1-p_m \right ) |0><0| + p_m |1><1|$$
$$F_1 = \left ( 1-p_m \right ) |1><1| + p_m |0><0|$$
#### p_M is probabilty of measurement error

#### Description 2:  Apply following mapping to qubit,
#### then perfect measurment in the (|0>, |1>) basis

$$p \to p' = \left ( 1-p_m \right )\ p + p_MXpX $$


#### Now we measure perfectly in the (|0>, |1>) basis

#### Both models give the same probabilties.

#### To measure perfectly in the  (|0>, |1>) basis, 
#### we define
$A_0 = |0><0|$ and $A_1 = |1><1|$
#### as measurement projects onto  (|0>, |1>) basis, we find

$$Tr(F_0 p)=\left( 1-p_M \right)Tr\left( A_0 p \right))+ p_MTr(A_1p)),$$
$$Tr(F_1 p)=\left( 1-p_M \right)Tr\left( A_1 p \right))+ p_MTr(A_0p)),$$
$$
$$

$$Tr(A_0 p')=\left( 1-p_M \right)Tr\left(A_0 p \right))+ p_MTr(XA_0 X_p))$$
$$= \left( 1-p_M \right)Tr\left(A_0 p \right))+ p_MTr(A_1p))$$
$$$$
$$Tr(A_1 p')=\left( 1-p_M \right)Tr\left(A_1 p \right))+ p_MTr(XA_1 X_p))$$
$$= \left( 1-p_M \right)Tr\left(A_1 p \right))+ p_MTr(A_0p))$$

#### For any given p_M, all methods here result in the same probabilites
#### Main model difference: State the measured qubit is projected to
#### When using POVM's, collapsed state of qubit is given by:

$p\to M_ipM_i^{\dagger}/Tr\left( F_ip \right)$  $i=0,1$
#### where,

$$M_0=\sqrt{1-p_M} |0><0| + \sqrt{p_M}|1><1|,$$
$$M_1=\sqrt{1-p_M} |1><1| + \sqrt{p_M}|0><0|$$

#### So, resulting state won't be initalialized in a known state
#### If description 2 is used, system initialized to |0> or |1>  depending on used project

#### Example: 
#### Consider 
$$ (1/\sqrt{2})\left( |0>+|1> \right)$$
#### Using both models in (19), probabilty of measuring |0> is 1/2


#### Else if POVM is used, state after m|easurement is: 
$$\sqrt{\left( 1-p_M \right)}|0> + \sqrt{p_m}|1>$$

#### Note to self: Page talks about using second model with projector A_0, state after measurement is |0>.


#### Measurement happens when discarding or reinitializing qubit to its known state
#### So, when a qubit is lost, it's removed from system
#### This is why we have our tracing function,
$$ Tr_i\left( p \right)$$
#### Loss of physical object ---> Cannot be measured or coupled to ancillary systems

## IV. The 3-Qubit Code: A good starting point for quantum error correction

## V. The 9-Qubit Code: The first full quantum code

## Vl. Quantum Error Detection

## Vll. Stabiliser Formalism

## Vlll. Quantum Error Correction with Stabiliser Codes

## lX. Digitization of Quantum Errors

## X. Fault Tolerant Quantum Error Correction and the Threashold Theorem

## Xl. Fault Tolerant Operations on Encoded Data

## Xll. Fault Tolerant Gate Design for Logical State Preparation

## Xlll. Loss Protection

## XlV. Some Modern Develpments in Quantum Error Correction